데이터 전처리/시각화 3-2 이론

​

0. 오늘 수업의 목표

수업이 끝나면 학습자는 아래를 “말로 설명”할 수 있어야 합니다.

GroupBy가 뭔지: “기준으로 묶고(그룹) → 요약값(집계)을 만든다”

Merge가 뭔지: “두 테이블을 키로 붙여서(조인) 정보 확장한다”

문자열/시간 처리가 왜 필요한지: “분석 가능한 형태로 통일하고 파생 피처를 만든다”

apply/map의 차이: “간단 치환(map) vs 행 단위 규칙(apply)”

# 1. 큰 그림: 원본 로그 → 리포트 테이블

## 데이터 분석에서 자주 보는 3단 구조

1. 원본 로그(거래/주문 단위)

- 주문 1건, 결제 1건처럼 “사건 기록”이 한 행씩 쌓인 데이터

- 행이 많고 값이 들쭉날쭉해서 그대로는 리포트(요약표) 만들기 어려움

2. 분석용 테이블(정제/통일/파생)

- 분석이 되도록 형식과 타입을 통일한 상태

- 예: 날짜 형식 통일, 가격을 숫자로 변환, 결제 여부(True/False) 통일

- 추가로 리포트에 필요한 파생 컬럼 생성(예: 연-월, 요일, 매출 컬럼)

3. 요약 테이블(집계/피벗)

- GroupBy/피벗으로 만든 최종 결과표

- 보고서나 대시보드에 바로 붙여 넣을 수 있는 형태

카페 매출 데이터로 치면

원본 로그 예시 컬럼

date, menu, price, qty, paid, store, channel ...

이 원본을 그대로 쓰면 문제가 생깁니다.

"4,500원" 같은 문자열 때문에 합계/평균 계산이 깨지거나

"Latte", "latte", " Latte "가 서로 다른 메뉴로 집계되어 결과가 왜곡됩니다.

그래서 먼저 분석용 테이블로 바꿉니다.

date → datetime

price, qty → 숫자

paid → True/False

ym(연-월), day_name(요일), sales(매출) 같은 파생 컬럼 생성

최종 요약표(리포트) 예시

월별 매출 추이: 이번 달 vs 지난 달 비교

요일별 매출 패턴: 어떤 요일이 강한지(운영/인력 배치에 유용)

메뉴 TOP / 카테고리별 매출: 잘 팔리는 메뉴/카테고리 파악(메뉴 전략)

결제 성공률(또는 실패율): 오류/취소/실패 비중 점검

# 2. GroupBy(그룹화) 이론

## 2-1. GroupBy란?

GroupBy = “기준 컬럼으로 묶고(그룹) → 각 그룹의 요약값(집계)을 만든다” 입니다.

원본 데이터가 주문 1건씩 쌓여 있을 때, 이를 리포트용 요약표로 바꾸는 가장 대표적인 방법이에요.

예시(카페 매출)

- 메뉴별 매출 합계: “Americano가 총 얼마 벌었나?”

- 매장별 주문 수: “광교점은 주문이 몇 건인가?”

- 월별 결제 성공률: “이번 달 결제 실패가 늘었나?”

In [1]:
import pandas as pd

# 1) 주문(원본 로그) 데이터: "주문 1건 = 행 1개"
df = pd.DataFrame([
    {"date": "2026-01-01", "store": "광교점", "menu": "Americano", "sales": 9000, "paid": True},
    {"date": "2026-01-01", "store": "광교점", "menu": "Latte",     "sales": 5000, "paid": True},
    {"date": "2026-01-02", "store": "광교점", "menu": "Latte",     "sales": 0,    "paid": False},
    {"date": "2026-01-03", "store": "수원점", "menu": "Americano", "sales": 4500, "paid": True},
    {"date": "2026-01-03", "store": "수원점", "menu": "Mocha",     "sales": 0,    "paid": False},
])

# 날짜를 datetime으로 바꾸고, 월(ym) 컬럼 생성 (월별 그룹화용)
df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str) # .dt = datetype

print("원본 로그")
print(df)


원본 로그
        date store       menu  sales   paid       ym
0 2026-01-01   광교점  Americano   9000   True  2026-01
1 2026-01-01   광교점      Latte   5000   True  2026-01
2 2026-01-02   광교점      Latte      0  False  2026-01
3 2026-01-03   수원점  Americano   4500   True  2026-01
4 2026-01-03   수원점      Mocha      0  False  2026-01


In [2]:
df.dtypes

date     datetime64[ns]
store            object
menu             object
sales             int64
paid               bool
ym               object
dtype: object

In [3]:

# 2) 메뉴별 매출 합계: "Americano가 총 얼마 벌었나?"
menu_sales = df.groupby("menu")["sales"].sum().reset_index()
print("\n메뉴별 매출 합계")
print(menu_sales)



메뉴별 매출 합계
        menu  sales
0  Americano  13500
1      Latte   5000
2      Mocha      0


In [4]:
type(menu_sales)

pandas.core.frame.DataFrame

In [5]:

# 3) 매장별 주문 수: "광교점은 주문이 몇 건인가?"
store_orders = df.groupby("store")["menu"].count().reset_index(name="orders")
print("\n매장별 주문 수")
print(store_orders)



매장별 주문 수
  store  orders
0   광교점       3
1   수원점       2


In [6]:
# 4) 월별 결제 성공률: "이번 달 결제 실패가 늘었나?"
# paid가 True/False이면 mean()이 성공률(비율)처럼 동작합니다.
monthly_paid_rate = df.groupby("ym")["paid"].mean().reset_index(name="paid_rate")
print("\n월별 결제 성공률")
print(monthly_paid_rate)


월별 결제 성공률
        ym  paid_rate
0  2026-01        0.6


2-2. 왜 GroupBy가 필요한가?

원본 로그는 대부분 “행 단위 사건” 이라서, 한 줄을 봐서는 흐름이 안 보입니다.

주문 1건을 봐서는 “이번 달 매출이 좋은지” 판단 불가

수백~수천 건을 “사람 눈”으로 봐서는 패턴을 찾기 어려움

그래서 GroupBy로 요약 단위를 바꿉니다.

주문 단위 → 메뉴 단위 / 매장 단위 / 월 단위

이렇게 바꾸면 “비교·추세·순위”가 가능해지고, 리포트가 됩니다.

In [7]:
import pandas as pd

# 원본 로그(주문 1건 = 1행)
df = pd.DataFrame([
    {"date": "2026-01-01", "store": "광교점", "menu": "Americano", "sales": 9000},
    {"date": "2026-01-01", "store": "광교점", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-02", "store": "광교점", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-03", "store": "수원점", "menu": "Americano", "sales": 4500},
    {"date": "2026-01-03", "store": "수원점", "menu": "Mocha",     "sales": 5500},
])

# 날짜 -> datetime, 월(ym) 컬럼 생성
df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str)

print(" 원본 로그(주문 1건 단위):")
print(df)

# 1) 주문 단위 -> 메뉴 단위 (메뉴별 매출 합계)
menu_report = df.groupby("menu")["sales"].sum().reset_index(name="total_sales")
print("\n GroupBy 결과(메뉴 단위) = 메뉴별 매출 합계:")
print(menu_report)

# 2) 주문 단위 -> 매장 단위 (매장별 매출 합계)
store_report = df.groupby("store")["sales"].sum().reset_index(name="total_sales")
print("\n GroupBy 결과(매장 단위) = 매장별 매출 합계:")
print(store_report)

# 3) 주문 단위 -> 월 단위 (월별 매출 합계)
month_report = df.groupby("ym")["sales"].sum().reset_index(name="total_sales")
print("\n GroupBy 결과(월 단위) = 월별 매출 합계:")
print(month_report)

# (추가) 순위/비교가 쉬워짐: 메뉴 매출 TOP 정렬
top_menu = menu_report.sort_values("total_sales", ascending=False)
print("\n 메뉴 매출 TOP(정렬로 순위 확인):")
print(top_menu)

 원본 로그(주문 1건 단위):
        date store       menu  sales       ym
0 2026-01-01   광교점  Americano   9000  2026-01
1 2026-01-01   광교점      Latte   5000  2026-01
2 2026-01-02   광교점      Latte   5000  2026-01
3 2026-01-03   수원점  Americano   4500  2026-01
4 2026-01-03   수원점      Mocha   5500  2026-01

 GroupBy 결과(메뉴 단위) = 메뉴별 매출 합계:
        menu  total_sales
0  Americano        13500
1      Latte        10000
2      Mocha         5500

 GroupBy 결과(매장 단위) = 매장별 매출 합계:
  store  total_sales
0   광교점        19000
1   수원점        10000

 GroupBy 결과(월 단위) = 월별 매출 합계:
        ym  total_sales
0  2026-01        29000

 메뉴 매출 TOP(정렬로 순위 확인):
        menu  total_sales
0  Americano        13500
1      Latte        10000
2      Mocha         5500


2-3. 집계 함수(aggregation) 핵심 5종

GroupBy의 결과를 만들 때는 “어떤 요약값을 뽑을지”가 중요합니다.

- sum: 총합

예: 총매출, 총수량

- mean: 평균

예: 평균 객단가, 평균 수량, 성공률(비율)

- count: 개수

예: 주문 건수(행 개수)

- nunique: 고유값 개수 -> Disticnt

예: 고유 고객 수, 고유 메뉴 수

- min / max: 최소/최대

예: 최소/최대 매출일, 최대 주문수량

초보자 포인트(중요)

paid가 True/False일 때, mean(paid)는 성공률처럼 해석될 수 있습니다.

(True는 1, False는 0처럼 동작하기 때문)

In [8]:
import pandas as pd

# 예시 데이터(주문 1건 = 1행)
df = pd.DataFrame([
    {"store": "광교점", "menu": "Americano", "qty": 2, "sales": 9000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "qty": 1, "sales": 5000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "qty": 2, "sales": 0,    "paid": False},
    {"store": "수원점", "menu": "Americano", "qty": 1, "sales": 4500, "paid": True},
    {"store": "수원점", "menu": "Mocha",     "qty": 1, "sales": 0,    "paid": False},
])

print("원본 데이터")
print(df)

# GroupBy + 집계 함수 5종 (매장별로 요약표 만들기)
summary = df.groupby("store").agg(
    total_sales=("sales", "sum"),     # sum: 총합
    avg_sales=("sales", "mean"),      # mean: 평균
    orders=("menu", "count"),         # count: 개수(행 개수)
    unique_menus=("menu", "nunique"), # nunique: 고유값 개수
    min_qty=("qty", "min"),           # min: 최소
    max_qty=("qty", "max"),           # max: 최대
    paid_rate=("paid", "mean"),       # mean(True/False) = 성공률(비율)
).reset_index()

print("\n매장별 요약(집계 5종 + paid_rate)")
print(summary)

# 참고: paid_rate가 왜 비율이 되는지 확인 (True=1, False=0처럼 동작)
print("\npaid 컬럼 확인(참고): True/False 평균 = 성공률")
print(df.groupby("store")["paid"].apply(list))

원본 데이터
  store       menu  qty  sales   paid
0   광교점  Americano    2   9000   True
1   광교점      Latte    1   5000   True
2   광교점      Latte    2      0  False
3   수원점  Americano    1   4500   True
4   수원점      Mocha    1      0  False

매장별 요약(집계 5종 + paid_rate)
  store  total_sales    avg_sales  orders  unique_menus  min_qty  max_qty  \
0   광교점        14000  4666.666667       3             2        1        2   
1   수원점         4500  2250.000000       2             2        1        1   

   paid_rate  
0   0.666667  
1   0.500000  

paid 컬럼 확인(참고): True/False 평균 = 성공률
store
광교점    [True, True, False]
수원점          [True, False]
Name: paid, dtype: object


2-4. 단일 기준 vs 다중 기준 (리포트가 달라짐)

1. 단일 기준 그룹화

기준: menu 하나만

결과: “메뉴별 매출 TOP”처럼 단순하고 직관적인 표

2. 다중 기준 그룹화

기준: ym(월) + day_name(요일)처럼 2개 이상

결과: “월별-요일별 매출 패턴” 같은 리포트형 분석이 가능

예: “1월은 월요일이 강하고, 금요일은 약하다” 같은 패턴

In [9]:
import pandas as pd

# 예시 데이터(주문 1건 = 1행)
df = pd.DataFrame([
    {"date": "2026-01-01", "menu": "Americano", "sales": 9000},
    {"date": "2026-01-02", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-03", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-06", "menu": "Americano", "sales": 4500},
    {"date": "2026-01-07", "menu": "Mocha",     "sales": 5500},
])

# 날짜 처리 + 파생 컬럼(월/요일) 만들기
df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str)     # "2026-01"
df["day_name"] = df["date"].dt.day_name()               # Monday, Tuesday...

print("원본 로그")
print(df)

# 1) 단일 기준 그룹화: menu 하나만
menu_report = (
    df.groupby("menu")["sales"].sum()
      .reset_index(name="total_sales")
      .sort_values("total_sales", ascending=False)
)

print("\n 단일 기준(menu) 그룹화: 메뉴별 매출 TOP")
print(menu_report)

# 2) 다중 기준 그룹화: ym(월) + day_name(요일)
ym_day_report = (
    df.groupby(["ym", "day_name"])["sales"].sum()
      .reset_index(name="total_sales")
      .sort_values(["ym", "total_sales"], ascending=[True, False])
)

print("\n 다중 기준(ym + day_name) 그룹화: 월별-요일별 매출")
print(ym_day_report)

# (선택) 리포트처럼 보이게 피벗 형태로 변환
pivot = (
    ym_day_report.pivot(index="ym", columns="day_name", values="total_sales")
    .fillna(0)
)

print("\n 피벗(월 x 요일) = 매출 패턴 표")
print(pivot)

원본 로그
        date       menu  sales       ym   day_name
0 2026-01-01  Americano   9000  2026-01   Thursday
1 2026-01-02      Latte   5000  2026-01     Friday
2 2026-01-03      Latte   5000  2026-01   Saturday
3 2026-01-06  Americano   4500  2026-01    Tuesday
4 2026-01-07      Mocha   5500  2026-01  Wednesday

 단일 기준(menu) 그룹화: 메뉴별 매출 TOP
        menu  total_sales
0  Americano        13500
1      Latte        10000
2      Mocha         5500

 다중 기준(ym + day_name) 그룹화: 월별-요일별 매출
        ym   day_name  total_sales
2  2026-01   Thursday         9000
4  2026-01  Wednesday         5500
0  2026-01     Friday         5000
1  2026-01   Saturday         5000
3  2026-01    Tuesday         4500

 피벗(월 x 요일) = 매출 패턴 표
day_name  Friday  Saturday  Thursday  Tuesday  Wednesday
ym                                                      
2026-01     5000      5000      9000     4500       5500


In [10]:
# # 1) 원본(세로형, Long format) — “주문/기록이 행으로 쌓이는 형태”
# +---------+-----------+-------+
# | Month   | Menu      | Sales |
# +---------+-----------+-------+
# | 2025-01 | Latte     |   1   |
# | 2025-01 | Americano |   2   |
# | 2025-01 | Mocha     |   3   |
# | 2025-02 | Latte     |   2   |
# | 2025-02 | Americano |   3   |
# | 2025-02 | Mocha     |   4   |
# +---------+-----------+-------+
# 
# # 2) 피벗 결과(가로형, Wide format) — “행/열로 보기 좋게 펼친 형태”
# +-----------+---------+---------+
# | Menu\Month| 2025-01 | 2025-02 |
# +-----------+---------+---------+
# | Latte     |   1     |   2     |
# | Americano |   2     |   3     |
# | Mocha     |   3     |   4     |
# +-----------+---------+---------+

## 2-5. agg()가 중요한 이유

초보자들이 흔히 “sum만” 만들고 끝내는데, 실무 리포트는 보통 한 표에 여러 지표가 필요합니다.

예)

total_sales(매출 합계)

orders(주문 수)

paid_rate(결제 성공률)

이런 걸 한 번에 뽑아주는 것이 agg()의 강점입니다.

즉, agg()는 리포트형 요약테이블 제작에 최적이에요.

In [11]:
import pandas as pd

# 예시 데이터(주문 1건 = 1행)
df = pd.DataFrame([
    {"store": "광교점", "menu": "Americano", "sales": 9000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "sales": 5000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "sales": 0,    "paid": False},
    {"store": "수원점", "menu": "Americano", "sales": 4500, "paid": True},
    {"store": "수원점", "menu": "Mocha",     "sales": 0,    "paid": False},
])

print("원본 로그")
print(df)

# 실무 리포트용: agg()로 여러 지표를 한 번에 만들기 (매장별 요약표)
report = df.groupby("store").agg(
    total_sales=("sales", "sum"),   # 매출 합계
    orders=("menu", "count"),       # 주문 수(행 개수)
    paid_rate=("paid", "mean"),     # 결제 성공률(True/False 평균)
).reset_index()

print("\n 매장별 리포트 요약표(agg로 한 번에 생성)")
print(report)

원본 로그
  store       menu  sales   paid
0   광교점  Americano   9000   True
1   광교점      Latte   5000   True
2   광교점      Latte      0  False
3   수원점  Americano   4500   True
4   수원점      Mocha      0  False

 매장별 리포트 요약표(agg로 한 번에 생성)
  store  total_sales  orders  paid_rate
0   광교점        14000       3   0.666667
1   수원점         4500       2   0.500000


2-6. MultiIndex가 뭐고 왜 불편한가?

다중 기준 GroupBy를 하면 결과가 종종 MultiIndex(계층 인덱스) 로 나옵니다.

장점: “계층 구조”를 표현할 수 있음

단점(초보자 체감):

표처럼 다루기가 어렵고

merge/pivot/저장(csv) 같은 후처리에 불편함

그래서 실무에서는 보통

reset_index()로 인덱스를 컬럼으로 내려서

다시 “일반적인 표 형태”로 만들어 씁니다.

In [12]:
import pandas as pd

# 예시 데이터
df = pd.DataFrame([
    {"date": "2026-01-01", "store": "광교점", "menu": "Americano", "sales": 9000},
    {"date": "2026-01-01", "store": "광교점", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-02", "store": "광교점", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-03", "store": "수원점", "menu": "Americano", "sales": 4500},
])

df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str)

print("원본 로그")
print(df)

# 다중 기준 GroupBy → 결과가 MultiIndex로 나올 수 있음(계층 인덱스)
multi = df.groupby(["ym", "store"])["sales"].sum()
print("\n 다중 기준 GroupBy 결과(Series, MultiIndex):")
print(multi)
print("\n인덱스 형태 확인:", type(multi.index))

# 실무에서 자주 하는 처리: reset_index()로 '표 형태'로 바꾸기
flat = multi.reset_index(name="total_sales")
print("\n reset_index() 후(일반적인 표 형태):")
print(flat)

# (참고) 이렇게 표가 되면 저장/merge/pivot이 쉬워짐
# flat.to_csv("summary.csv", index=False)  # 저장도 쉬움

원본 로그
        date store       menu  sales       ym
0 2026-01-01   광교점  Americano   9000  2026-01
1 2026-01-01   광교점      Latte   5000  2026-01
2 2026-01-02   광교점      Latte   5000  2026-01
3 2026-01-03   수원점  Americano   4500  2026-01

 다중 기준 GroupBy 결과(Series, MultiIndex):
ym       store
2026-01  광교점      19000
         수원점       4500
Name: sales, dtype: int64

인덱스 형태 확인: <class 'pandas.core.indexes.multi.MultiIndex'>

 reset_index() 후(일반적인 표 형태):
        ym store  total_sales
0  2026-01   광교점        19000
1  2026-01   수원점         4500


3. 피벗(표 형태 변환) 이론: “리포트용 표 만들기” 

3-1. 피벗이 필요한 순간

피벗은 한마디로 **“리포트에서 보기 좋은 표 모양으로 바꾸는 작업”**입니다.

원본 로그는 보통 “주문 1건 = 1행”이라 아래처럼 생겼죠.

(원본) date, menu, store, sales ... 형태

이 상태에서는 “월별·요일별 패턴”을 한눈에 보기 어렵습니다.

하지만 보고서(리포트) 표는 보통 이렇게 생깁니다.

행(Row): 월(예: 2026-01, 2026-02 …)

열(Column): 요일(예: Mon, Tue …)

값(Value): 매출(합계)

이렇게 만들면 무엇이 좋아지냐면,

“어느 요일이 강한지”가 한눈에 보이고

월별로 요일 패턴이 바뀌는지도 바로 비교가 됩니다.

중요 포인트는:

이런 “월 × 요일” 표는 원본 로그에서 바로 나오지 않고,

먼저 **GroupBy로 요약(월+요일별 매출 합계)**을 만든 다음,

그 결과를 피벗으로 표 모양을 바꿔서 완성합니다.

In [13]:
import pandas as pd

# 1) 원본 로그(주문 1건 = 1행)
df = pd.DataFrame([
    {"date": "2026-01-01", "menu": "Americano", "store": "광교점", "sales": 9000},
    {"date": "2026-01-02", "menu": "Latte",     "store": "광교점", "sales": 5000},
    {"date": "2026-01-03", "menu": "Latte",     "store": "수원점", "sales": 5000},
    {"date": "2026-01-06", "menu": "Americano", "store": "광교점", "sales": 4500},
    {"date": "2026-02-03", "menu": "Mocha",     "store": "수원점", "sales": 5500},
])

# 2) 날짜 처리 + 파생 컬럼(월/요일) 만들기
df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str)   # 예: "2026-01"
df["day"] = df["date"].dt.day_name()                  # 예: "Monday"

print(" 원본 로그(주문 단위)")
print(df[["date", "ym", "day", "store", "menu", "sales"]])

# 3) (핵심) 월+요일별 매출 합계로 먼저 요약(GroupBy)
summary = (
    df.groupby(["ym", "day"])["sales"].sum()
      .reset_index(name="total_sales")
)

print("\n GroupBy 요약(월+요일별 매출 합계)")
print(summary)

# 4) (핵심) 피벗으로 '월 x 요일' 리포트 표 만들기
report = (
    summary.pivot(index="ym", columns="day", values="total_sales")
    .fillna(0)  # 데이터 없는 칸은 0으로
)

print("\n 피벗 리포트(월 x 요일 매출 표)")
print(report)

 원본 로그(주문 단위)
        date       ym       day store       menu  sales
0 2026-01-01  2026-01  Thursday   광교점  Americano   9000
1 2026-01-02  2026-01    Friday   광교점      Latte   5000
2 2026-01-03  2026-01  Saturday   수원점      Latte   5000
3 2026-01-06  2026-01   Tuesday   광교점  Americano   4500
4 2026-02-03  2026-02   Tuesday   수원점      Mocha   5500

 GroupBy 요약(월+요일별 매출 합계)
        ym       day  total_sales
0  2026-01    Friday         5000
1  2026-01  Saturday         5000
2  2026-01  Thursday         9000
3  2026-01   Tuesday         4500
4  2026-02   Tuesday         5500

 피벗 리포트(월 x 요일 매출 표)
day      Friday  Saturday  Thursday  Tuesday
ym                                          
2026-01  5000.0    5000.0    9000.0   4500.0
2026-02     0.0       0.0       0.0   5500.0


3-2. “긴 형태(long) → 넓은 형태(wide)” 감각 잡기

피벗을 이해할 때 가장 쉬운 관점은 이것입니다.

긴 형태(long): 행이 길게 늘어져 있는 형태

예: 2026-01, Mon, 120000

2026-01, Tue, 90000

2026-02, Mon, 150000 …

특징: “데이터 분석엔 편하지만”, 사람이 보기엔 패턴이 잘 안 보임

넓은 형태(wide): 열이 펼쳐져서 표처럼 보이는 형태

예:

행: 2026-01 / 2026-02

열: Mon, Tue, Wed…

값: 매출

특징: 보고서/대시보드에 바로 붙이기 좋음

초보자에게는 이렇게 말하면 직관적입니다.

In [14]:
import pandas as pd

# 1) 긴 형태(long) 데이터: "요일"이 행에 들어있는 형태
long_df = pd.DataFrame([
    {"ym": "2026-01", "day": "Mon", "sales": 120000},
    {"ym": "2026-01", "day": "Tue", "sales":  90000},
    {"ym": "2026-02", "day": "Mon", "sales": 150000},
    {"ym": "2026-02", "day": "Tue", "sales":  80000},
])

print(" 긴 형태(long) 데이터")
print(long_df)

#  2) 넓은 형태(wide)로 변환: 요일을 '열'로 펼치기(피벗)
wide_df = long_df.pivot(index="ym", columns="day", values="sales").fillna(0)

print("\n 넓은 형태(wide) 데이터 (월 x 요일 표)")
print(wide_df)

 긴 형태(long) 데이터
        ym  day   sales
0  2026-01  Mon  120000
1  2026-01  Tue   90000
2  2026-02  Mon  150000
3  2026-02  Tue   80000

 넓은 형태(wide) 데이터 (월 x 요일 표)
day         Mon    Tue
ym                    
2026-01  120000  90000
2026-02  150000  80000


3-3. pivot vs unstack은 뭐가 다른가?

둘 다 결과는 비슷하게 “표 형태(wide)”를 만들 수 있어요.

pivot

“행/열/값을 지정해서 피벗 표를 만든다”

초보자에게 가장 추천(가독성이 좋음)

unstack

GroupBy 결과가 MultiIndex일 때

“한 인덱스 레벨을 열로 펼친다” 느낌

pivot보다 조금 더 ‘GroupBy 결과를 다루는’ 방식에 가깝습니다.

In [15]:
import pandas as pd

# 예시 데이터(일부 조합이 반복되도록 일부러 2행 넣음: 2026-01 + Mon)
df = pd.DataFrame([
    {"ym": "2026-01", "day": "Mon", "sales": 100},
    {"ym": "2026-01", "day": "Mon", "sales": 50},   # 같은 조합이 2개라서 pivot이 바로 안 됨
    {"ym": "2026-01", "day": "Tue", "sales": 80},
    {"ym": "2026-02", "day": "Mon", "sales": 120},
    # 2026-02의 Tue가 없어서 피벗 후 NaN이 생길 수 있음
])

print("원본(long 형태)")
print(df)

# 실수 포인트 1) 같은 ym+day 조합이 여러 행이면 pivot이 바로 안 됨
# 해결: GroupBy로 먼저 1개 값(합계/평균 등)으로 요약
summary = df.groupby(["ym", "day"])["sales"].sum().reset_index(name="total_sales")

print("\nGroupBy로 먼저 요약(중복 조합 해결)")
print(summary)

# 1) pivot 방식: 행/열/값을 지정해서 표 만들기
pivot_table = summary.pivot(index="ym", columns="day", values="total_sales")
print("\npivot 결과")
print(pivot_table)

# 2) unstack 방식: GroupBy 결과(MultiIndex)를 열로 펼치기
multi = df.groupby(["ym", "day"])["sales"].sum()  # MultiIndex Series
unstack_table = multi.unstack()
print("\nunstack 결과")
print(unstack_table)

# 실수 포인트 2) 어떤 조합이 없으면 NaN이 생김
# 리포트에서는 보통 0으로 채움
pivot_filled = pivot_table.fillna(0)
print("\nNaN을 0으로 채운 리포트 표")
print(pivot_filled)

원본(long 형태)
        ym  day  sales
0  2026-01  Mon    100
1  2026-01  Mon     50
2  2026-01  Tue     80
3  2026-02  Mon    120

GroupBy로 먼저 요약(중복 조합 해결)
        ym  day  total_sales
0  2026-01  Mon          150
1  2026-01  Tue           80
2  2026-02  Mon          120

pivot 결과
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN

unstack 결과
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN

NaN을 0으로 채운 리포트 표
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   0.0


3-4. 피벗에서 자주 만나는 실수 포인트 2개 (확장 + 간단 예제코드)

1) 값이 여러 개라서 피벗이 안 되는 경우

피벗은 기본적으로 “한 칸(ym + day 조합) = 값 1개”여야 합니다.

그런데 원본 로그는 보통 주문이 여러 건이라 같은 월(ym) + 요일(day) 조합이 여러 번 등장합니다.

예)

2026-01 + Mon 주문이 3건이면

피벗에서 “2026-01 행의 Mon 열”에 값이 3개가 들어가려 해서 충돌이 납니다.

그래서 실무에서는 보통 이렇게 합니다.

1단계: GroupBy로 먼저 요약(sum/mean 등)해서 “칸당 값 1개”로 만들기

2단계: pivot으로 표 모양 만들기

간단 예제코드

In [16]:
import pandas as pd

df = pd.DataFrame([
    {"ym": "2026-01", "day": "Mon", "sales": 100},
    {"ym": "2026-01", "day": "Mon", "sales": 50},   # 같은 조합이 반복
    {"ym": "2026-01", "day": "Tue", "sales": 80},
    {"ym": "2026-02", "day": "Mon", "sales": 120},
])

# 바로 pivot을 시도하면 "한 칸에 여러 값"이라 에러가 날 수 있음
# 해결: 먼저 GroupBy로 한 칸당 값 1개(합계)로 요약
summary = df.groupby(["ym", "day"])["sales"].sum().reset_index(name="total_sales")

# 그 다음 pivot
report = summary.pivot(index="ym", columns="day", values="total_sales")

print(summary)
print(report)

        ym  day  total_sales
0  2026-01  Mon          150
1  2026-01  Tue           80
2  2026-02  Mon          120
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN


2) 빈 칸(NaN)이 생기는 것

피벗 표는 “월 × 요일”처럼 모든 칸이 항상 존재하지 않습니다.

예를 들어 2026-02에는 화요일(Tue) 매출 데이터가 아예 없다면, 해당 칸은 비게 되고 NaN이 됩니다.

이건 오류가 아니라 “그 조합의 데이터가 없다”는 뜻입니다.

다만 리포트에서는 보통 아래 중 하나로 처리합니다.

매출처럼 “없으면 0”이 자연스러우면 0으로 채움

결측을 의미 있게 해석해야 하면 NaN을 유지

간단 예제코드

In [17]:
import pandas as pd

summary = pd.DataFrame([
    {"ym": "2026-01", "day": "Mon", "total_sales": 150},
    {"ym": "2026-01", "day": "Tue", "total_sales": 80},
    {"ym": "2026-02", "day": "Mon", "total_sales": 120},
    # 2026-02의 Tue 데이터는 일부러 없음 -> 피벗 후 NaN 발생
])

report = summary.pivot(index="ym", columns="day", values="total_sales")

print("피벗 결과(빈 칸은 NaN)")
print(report)

report_filled = report.fillna(0)
print("\n리포트용(매출은 0으로 채움)")
print(report_filled)

피벗 결과(빈 칸은 NaN)
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN

리포트용(매출은 0으로 채움)
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   0.0


정리

NaN은 “값이 없다/기록이 없다”는 신호

매출 리포트는 보통 fillna(0)을 많이 사용

상황에 따라 NaN을 유지하는 것이 더 정확할 수도 있음(예: 측정 실패, 데이터 누락 분석)

한 문장 결론

*피벗은 “GroupBy로 만든 요약 결과를, 리포트에서 보기 좋은 표(행×열 구조)로 바꾸는 마지막 작업”입니다.

4. 문자열 처리 이론(실무형) 확장

4-1. 왜 문자열 처리가 필수인가?

실무 데이터에서 문자열은 거의 항상 “정리되지 않은 상태”로 들어옵니다.

문제는 이게 보기만 지저분한 게 아니라, 분석 결과를 틀리게 만들거나 계산 자체를 못 하게 만든다는 점입니다.

대표적인 상황 2가지

1. 같은 값인데 다르게 저장됨(표기 흔들림)

" Latte ", "latte", "LATTE" 사람 눈에는 같은 메뉴지만, 컴퓨터는 다른 값으로 봅니다.

결과: GroupBy를 하면 Latte가 여러 그룹으로 쪼개져서 집계가 깨집니다.

2. 숫자가 문자열로 섞여 들어옴(단위/쉼표/통화 기호)

- "4,500원", "5000", "5,000원" → 숫자처럼 보이지만 문자열이라 합계/평균 계산이 실패하거나 왜곡됩니다.

문자열 정리를 안 하면 생기는 문제

- 메뉴별 매출 TOP이 잘못 나옴(같은 메뉴가 여러 줄로 나뉨)

- 가격 합계가 안 되거나(문자열이라), 일부만 변환돼 왜곡됨

    - 필터 조건이 제대로 안 걸림(공백/대소문자 차이 때문에)

핵심 정리

문자열 처리는 “꾸미기”가 아니라 분석 가능한 값으로 통일하는 작업입니다.

즉, “분석 전에 통일”이 목적입니다.

4-2. 초보자 기준 핵심 5가지 + 예시코드

아래 코드는 카페 데이터를 예로 들어, 현업에서 가장 많이 쓰는 문자열 처리 패턴을 한 번에 보여줍니다.

In [18]:
import pandas as pd

df = pd.DataFrame([
    {"menu": " Latte ",        "price": "4,500원", "note": "NEW member"},
    {"menu": "latte",          "price": "5000",     "note": "vip MEMBER"},
    {"menu": "LATTE",          "price": "5,000원",  "note": "Member coupon"},
    {"menu": "Americano",      "price": "4500원",   "note": "guest"},
    {"menu": "Vanilla Latte",  "price": "5,800원",  "note": "new_member"},
])

print("원본")
print(df)

원본
            menu   price           note
0         Latte   4,500원     NEW member
1          latte    5000     vip MEMBER
2          LATTE  5,000원  Member coupon
3      Americano   4500원          guest
4  Vanilla Latte  5,800원     new_member


In [19]:
# 1) 공백 제거: strip
# 
# 앞뒤 공백은 그룹화/필터링을 망치는 대표 원인입니다.

df["menu_clean"] = df["menu"].str.strip() # _clean : 새로운 컬럼을 만들어 정제되었다고 표기! 항상 새로운 테이블 또는 파생 컬럼을 만들어 비교한다.
print("\nstrip 적용")
print(df[["menu", "menu_clean"]])


strip 적용
            menu     menu_clean
0         Latte           Latte
1          latte          latte
2          LATTE          LATTE
3      Americano      Americano
4  Vanilla Latte  Vanilla Latte


In [20]:
# 2) 대소문자 통일: lower / title
# 
# lower()는 비교/필터에 강함
# 
# title()은 보기(리포트 출력)에 깔끔함

df["menu_lower"] = df["menu_clean"].str.lower()
df["menu_title"] = df["menu_clean"].str.title()

print("\nlower/title 적용")
print(df[["menu_clean", "menu_lower", "menu_title"]])

# 실무 팁
# 
# - 내부 처리(정합성)는 lower()로 통일
# 
# - 최종 출력(보기)은 title()로 정리(선택)
# 
#     - 이 방식이 실수 줄이기 좋습니다.


lower/title 적용
      menu_clean     menu_lower     menu_title
0          Latte          latte          Latte
1          latte          latte          Latte
2          LATTE          latte          Latte
3      Americano      americano      Americano
4  Vanilla Latte  vanilla latte  Vanilla Latte


In [21]:
# 3) 포함 여부 필터: contains
# 
# 특정 단어가 들어간 행만 뽑을 때 사용합니다.

# note 컬럼에서 'member'가 포함된 행만 필터(대소문자 무시)
member_df = df[df["note"].str.contains("member", case=False, na=False)] # case => 대소문자
print("\ncontains로 member 관련만 필터")
print(member_df[["note"]])


contains로 member 관련만 필터
            note
0     NEW member
1     vip MEMBER
2  Member coupon
4     new_member


In [22]:
# 4) 치환: replace
# 
# 가격에서 쉼표/원 제거 후 숫자로 바꾸는 전형적인 패턴입니다.

df["price_num"] = (
    df["price"]
      .str.replace(",", "", regex=False)
      .str.replace("원", "", regex=False)
)

df["price_num"] = pd.to_numeric(df["price_num"], errors="coerce")

print("\nreplace + 숫자 변환")
print(df[["price", "price_num"]])


replace + 숫자 변환
    price  price_num
0  4,500원       4500
1    5000       5000
2  5,000원       5000
3   4500원       4500
4  5,800원       5800


In [23]:
# 5) 분리: split
# 
# 문자열을 나눠서 필요한 정보만 뽑을 때 사용합니다.
# 
# 예: 메뉴명을 공백 기준으로 나누고 첫 단어만 추출

df["menu_first_word"] = df["menu_title"].str.split().str[0]
print("\nsplit 적용(첫 단어 추출)")
print(df[["menu_title", "menu_first_word"]])


split 적용(첫 단어 추출)
      menu_title menu_first_word
0          Latte           Latte
1          Latte           Latte
2          Latte           Latte
3      Americano       Americano
4  Vanilla Latte         Vanilla


# 5. 시간 데이터 처리 이론(실무형)

5-1. 왜 날짜 처리가 중요한가?

리포트는 거의 항상 “시간축”이 들어갑니다.

월별 매출, 주간 매출, 요일별 패턴, 시간대별 피크 같은 분석은 전부 “시간”이 기준이에요. pandas도 시간 데이터 처리를 위한 기능을 많이 제공합니다. 

문제는 날짜가 문자열(str)로 들어오면 아래가 자주 깨집니다.

- 정렬이 이상해짐(문자열 정렬 문제)

문자열은 “시간 순서”가 아니라 “글자 순서(사전순)”로 정렬될 수 있습니다.

특히 dd/mm/yyyy 같은 형식은 시간 순서와 사전순이 잘 맞지 않아 더 자주 문제를 만듭니다. 

간단 예제코드: 문자열 정렬 vs datetime 정렬

In [24]:
import pandas as pd

df = pd.DataFrame({
    "date_str": ["2026/01/02", "2026/01/10", "2026/01/03"],
    "sales": [100, 200, 150]
})

print(df.sort_values("date_str"))  # 문자열 기준 정렬

df["date"] = pd.to_datetime(df["date_str"])
print(df.sort_values("date"))      # datetime 기준 정렬

     date_str  sales
0  2026/01/02    100
2  2026/01/03    150
1  2026/01/10    200
     date_str  sales       date
0  2026/01/02    100 2026-01-02
2  2026/01/03    150 2026-01-03
1  2026/01/10    200 2026-01-10


기준 정렬

1. 파생 피처를 못 만듦(월/요일/주차 등)

“월별/요일별” 분석을 하려면 month, day_name 같은 파생 컬럼이 필요합니다.

그런데 날짜가 문자열이면 .dt(datetime 전용 기능)를 쓸 수 없어서 파생 피처 생성이 막힙니다. 

2. 기간 필터링이 어려움

“2026-01만 보기”, “특정 기간만 자르기” 같은 작업이 날짜 계산 기반이라,

문자열 상태로는 실수(경계값, 포함/제외)가 늘어납니다.

## 5-2. 핵심 개념: “datetime으로 바꾸고 파생 피처 만든다”

초보자 기준으로는 이 한 줄이면 충분합니다.

- 문자열 날짜 → pd.to_datetime()로 변환

- .dt로 year/month/day_name 같은 파생 피처 생성

- (리포트용) ym(연-월) 컬럼 만들어 GroupBy/Pivot에 활용

pd.to_datetime()는 문자열/배열/Series 등을 pandas의 datetime 타입으로 바꿔주는 대표 함수입니다. 

간단 예제코드: datetime 변환 + 파생 피처 만들기

In [25]:
import pandas as pd

df = pd.DataFrame({
    "date": ["2026-01-01", "2026-01-03", "2026-02-01"],
    "sales": [10000, 15000, 20000]
})

df["date"] = pd.to_datetime(df["date"])  # 핵심 1) datetime 변환

df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day_name"] = df["date"].dt.day_name()

# 연-월(리포트에서 자주 쓰는 키): to_period('M') 활용
df["ym"] = df["date"].dt.to_period("M").astype(str)

print(df)

        date  sales  year  month  day_name       ym
0 2026-01-01  10000  2026      1  Thursday  2026-01
1 2026-01-03  15000  2026      1  Saturday  2026-01
2 2026-02-01  20000  2026      2    Sunday  2026-02


초보자용 요약

- datetime 변환 = 시간 분석의 문 열기

- 파생 피처(ym, day_name 등) = 월별/요일별 리포트의 재료

- 이후 GroupBy/Pivot이 훨씬 쉬워집니다.

# 6. 데이터 결합(merge/join/concat)

## 6-1. 결합은 왜 필요한가?

원본 로그(주문 데이터)에는 보통 분석에 필요한 정보가 다 들어있지 않습니다.

그래서 “다른 테이블의 정보를 옆으로 붙여서(확장해서)” 분석을 합니다.

카페 예시

- 주문 로그: date, menu, sales, customer_id 는 있는데

- 메뉴 카테고리(커피/라떼/디저트)는 menu_map 같은 별도 표에 있음

- 고객 도시/등급은 customers 표에 있음

즉, 분석을 하려면

- 주문 로그 + 메뉴 매핑표 + 고객 정보표를 결합해야 합니다.

간단 예제코드(데이터 준비)

In [26]:
import pandas as pd

orders = pd.DataFrame([
    {"order_id": 1, "date": "2026-01-01", "menu": "Latte",     "sales": 5000, "customer_id": "C01"},
    {"order_id": 2, "date": "2026-01-01", "menu": "Americano", "sales": 4500, "customer_id": "C02"},
    {"order_id": 3, "date": "2026-01-02", "menu": "Cake",      "sales": 6000, "customer_id": "C03"},
])

menu_map = pd.DataFrame([
    {"menu": "Latte",     "category": "Coffee"},
    {"menu": "Americano", "category": "Coffee"},
    # Cake가 빠져있다고 가정(매핑 누락 사례)
])

customers = pd.DataFrame([
    {"customer_id": "C01", "city": "Suwon", "grade": "VIP"},
    {"customer_id": "C02", "city": "Yongin","grade": "NEW"},
    {"customer_id": "C03", "city": "Suwon", "grade": "NORMAL"},
])

print(orders)
print(menu_map)
print(customers)

   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03
        menu category
0      Latte   Coffee
1  Americano   Coffee
  customer_id    city   grade
0         C01   Suwon     VIP
1         C02  Yongin     NEW
2         C03   Suwon  NORMAL


6-2. merge = “키로 붙이는 조인”

merge는 SQL JOIN과 비슷하게 공통 키(key)로 옆으로 붙이는 기능입니다. pandas에서도 “database-style join”이라고 설명합니다. 

예: menu를 키로 menu_map(메뉴→카테고리)를 붙이면

“메뉴별 매출”을 “카테고리별 매출” 같은 분석으로 확장할 수 있습니다.

간단 예제코드(주문 + 메뉴카테고리 붙이기)

In [27]:
merged = orders.merge(menu_map, on="menu", how="left")  # 매핑표는 보통 left로 붙임
print(merged)
# 이제 category가 생겼으니 카테고리별 매출도 가능해집니다.

cat_sales = merged.groupby("category")["sales"].sum()
print(cat_sales)

   order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee
2         3  2026-01-02       Cake   6000         C03      NaN
category
Coffee    9500
Name: sales, dtype: int64


6-3. how 옵션(초보자 필수 3종)

how는 “어떤 기준으로 행을 남길지”를 정하는 옵션이고, pandas 문서에 각 동작이 SQL join과 유사하다고 정리되어 있습니다. 

1. left: 왼쪽(원본) 유지

실무에서 “원본 로그는 유지하고, 매핑이 있으면 붙이고, 없으면 NaN으로 확인”할 때 가장 많이 씁니다. 

2. inner: 양쪽에 모두 있는 것만

“매핑이 제대로 됐는지(정합성)” 확인할 때 유용합니다. 

3. outer: 전체 합치기(누락 탐색)

“어느 쪽에만 있는 값이 있나?” 누락/신규 항목 탐색에 좋습니다. 

간단 예제코드(left / inner / outer 비교)

In [28]:
left_join  = orders.merge(menu_map, on="menu", how="left")
inner_join = orders.merge(menu_map, on="menu", how="inner")
outer_join = orders.merge(menu_map, on="menu", how="outer")

print("left\n", left_join)
print("inner\n", inner_join)
print("outer\n", outer_join)

left
    order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee
2         3  2026-01-02       Cake   6000         C03      NaN
inner
    order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee
outer
    order_id        date       menu  sales customer_id category
0         2  2026-01-01  Americano   4500         C02   Coffee
1         3  2026-01-02       Cake   6000         C03      NaN
2         1  2026-01-01      Latte   5000         C01   Coffee


실무 팁(초보자에게 꼭)

- 매핑표 붙일 때는 보통 left로 붙인 뒤,

- category가 NaN인 행을 찾아서 “매핑 누락”을 수정합니다.

간단 예제코드(매핑 누락 찾기: indicator 활용)

In [29]:
check = orders.merge(menu_map, on="menu", how="left", indicator=True)
missing = check[check["_merge"] == "left_only"]  # 왼쪽에만 있음 = 매핑표에 없음
print(missing[["menu", "order_id"]])

# - indicator=True는 병합 결과에 행 출처를 표시해 주는 옵션입니다. 

# - 참고로 DataFrame.join()은 기본이 index 기반 결합에 가깝고(기본적으로 인덱스로 조인), 기본 how='left'입니다. 

   menu  order_id
2  Cake         3


6-4. concat은 뭐가 다르냐?

concat은 merge처럼 “키로 옆에 붙이기”가 아니라, 보통 위아래로 쌓아서(누적) 데이터 양을 늘릴 때 씁니다.

pandas 공식 문서도 “특정 axis 방향으로 concatenate(연결)”한다고 설명합니다. 

느낌 정리

- merge: 옆으로 붙이기(정보 확장)

- concat: 위아래로 쌓기(데이터 누적)

간단 예제코드(두 달의 주문 로그를 위아래로 누적)

In [30]:
jan = pd.DataFrame([
    {"order_id": 1, "ym": "2026-01", "sales": 5000},
    {"order_id": 2, "ym": "2026-01", "sales": 4500},
])

feb = pd.DataFrame([
    {"order_id": 3, "ym": "2026-02", "sales": 6000},
])

all_orders = pd.concat([jan, feb], ignore_index=True)  # 위아래로 쌓기
print(all_orders)
# (참고) concat은 axis=1로 “옆으로” 붙일 수도 있지만, 그 경우는 같은 인덱스 기준으로 나란히 놓는 성격이라 보통은 merge와 용도가 다릅니다. 

   order_id       ym  sales
0         1  2026-01   5000
1         2  2026-01   4500
2         3  2026-02   6000


7. apply vs map 이론(언제 뭘 쓰나) 확장 + 쉬운 예제코드

7-1. map: “값 치환/매핑”에 특화(간단·빠름)

map은 한 컬럼의 값들을 “딕셔너리/매핑표”로 바꿀 때 가장 깔끔합니다.

즉, “A를 B로 바꿔라” 같은 1:1 치환 작업이면 map이 1순위예요.

대표 사용처

- 채널 코드 바꾸기: kiosk → 키오스크, app → 앱

- 등급 코드 바꾸기: 1 → VIP, 2 → NORMAL 등

- 메뉴/카테고리 매핑(간단 버전): Latte → Coffee

간단 예제코드: channel 코드 치환


In [31]:
import pandas as pd

df = pd.DataFrame({
    "channel": ["kiosk", "app", "kiosk", "web"]
})

channel_map = {
    "kiosk": "키오스크",
    "app": "앱",
    "web": "웹"
}

df["channel_name"] = df["channel"].map(channel_map)
print(df)
# 초보자 실수 포인트
# 
# - 매핑표에 없는 값은 결과가 NaN이 됩니다.
# 
# - 누락이 있을 수 있으니 확인하는 습관이 좋아요.
# 
# 누락 확인 예시

missing = df[df["channel_name"].isna()]
print(missing)

  channel channel_name
0   kiosk         키오스크
1     app            앱
2   kiosk         키오스크
3     web            웹
Empty DataFrame
Columns: [channel, channel_name]
Index: []


7-2. apply: “행 단위 규칙”이 필요할 때

apply는 규칙(로직)을 직접 써야 할 때 씁니다.

특히 이런 상황에서 유용합니다.

- 숫자 구간에 따라 등급 분류: 매출 0이면 C, 1~9999면 B, 10000 이상이면 A

- 여러 컬럼을 동시에 보고 판단: paid가 False면 무조건 "FAIL", True면 매출 구간으로 등급 등



In [32]:
# 간단 예제코드 1: 매출 구간 등급(단일 컬럼 기준)
import pandas as pd

df = pd.DataFrame({"sales": [0, 3000, 12000]})

def grade_sales(x):
    if x == 0:
        return "C"
    elif x < 10000:
        return "B"
    else:
        return "A"

df["grade"] = df["sales"].apply(grade_sales)
print(df)
# 간단 예제코드 2: 여러 컬럼을 보고 판단(행 단위)

import pandas as pd

df = pd.DataFrame([
    {"sales": 12000, "paid": True},
    {"sales": 5000,  "paid": True},
    {"sales": 8000,  "paid": False},
])

def final_label(row):
    if row["paid"] is False:
        return "FAIL"
    if row["sales"] >= 10000:
        return "A"
    return "B"

df["label"] = df.apply(final_label, axis=1)  # axis=1: 행 기준
print(df)

   sales grade
0      0     C
1   3000     B
2  12000     A
   sales   paid label
0  12000   True     A
1   5000   True     B
2   8000  False  FAIL


7-3. 초보자에게 꼭 알려야 할 점: apply는 편하지만 느릴 수 있음

apply는 “파이썬 함수로 한 행씩 처리”하는 경우가 많아서 데이터가 커지면 느려질 수 있습니다.

그래서 가능하면 아래처럼 **벡터화(조건식)**로 처리하는 습관이 좋아요.

apply 대신 벡터화로 등급 만들기(빠르고 깔끔)

In [33]:
import pandas as pd

df = pd.DataFrame({"sales": [0, 3000, 12000]})

df["grade_vec"] = "B"
df.loc[df["sales"] == 0, "grade_vec"] = "C"
df.loc[df["sales"] >= 10000, "grade_vec"] = "A"

print(df)

   sales grade_vec
0      0         C
1   3000         B
2  12000         A


한 줄 결론

map = 한 컬럼 값들을 딕셔너리로 치환할 때(가볍고 빠름)

apply = 규칙이 필요하거나 여러 컬럼을 같이 보고 판단해야 할 때(편하지만 커지면 느릴 수 있음)

8. 오늘 수업을 한 문장으로 정리

“원본 로그를 문자열/날짜 정리로 계산 가능하게 만들고, GroupBy로 요약하고, Merge로 정보 확장해서, 리포트용 표(요약테이블)를 만든다.”

9. 초보자용 퀴즈(이론 체크, 5문항)

GroupBy는 왜 필요한가? “원본 로그”와 비교해서 설명해보세요.

paid_rate = mean(paid)가 “비율”이 되는 이유는?

MultiIndex가 불편한 이유와 해결 방법 1가지는?

merge의 left와 inner의 차이를 “매핑 누락 찾기” 관점에서 설명해보세요.

map과 apply는 언제 각각 쓰는가? 예시 1개씩 들어보세요.